In [21]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [22]:
spark.version

'3.3.2'

In [23]:
spark

In [24]:
!gunzip -c fhv_tripdata_2019-10.csv.gz > fhvhv_tripdata_2019_10.csv

In [ ]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2019_10.csv')

In [25]:
!head -n 1001 fhvhv_tripdata_2019_10.csv > head.csv

In [26]:
import pandas as pd
df_pandas = pd.read_csv('head.csv')

In [27]:
df_pandas.head(5)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264.0,264.0,NaN,B00009
1,B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264.0,264.0,NaN,B00013
2,B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264.0,264.0,NaN,B00014
3,B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264.0,264.0,NaN,B00014
4,B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264.0,264.0,NaN,B00014


In [36]:
df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [37]:
spark.createDataFrame(df_pandas[df_pandas['Affiliated_base_number'].notnull()]).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [30]:
from pyspark.sql import types

In [31]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [32]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2019_10.csv')

In [34]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   null|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   null|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   null|                B00014|
|     B00021         |2019-10-01 00:00:4

In [39]:
type(df)

pyspark.sql.dataframe.DataFrame

In [40]:
df.repartition(6).write.parquet('fhvhv/2019/10/')

In [41]:
df.registerTempTable('fhv_data')

/home/ubuntu/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [42]:
df_result = spark.sql("""
SELECT count(*) FROM
    fhv_data
where pickup_datetime between '2019-10-15 00:00:00' and '2019-10-15 23:59:59'
""")

In [43]:
df_result.show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



In [47]:
df_longest = spark.sql("""
SELECT pickup_datetime, dropOff_datetime , (dropOff_datetime - pickup_datetime) as duration FROM
    fhv_data
order by duration desc limit 1
""").show()

+-------------------+-------------------+--------------------+
|    pickup_datetime|   dropOff_datetime|            duration|
+-------------------+-------------------+--------------------+
|2019-10-11 18:00:00|2091-10-11 18:30:00|INTERVAL '26298 0...|
+-------------------+-------------------+--------------------+



In [48]:
df_trip = spark.sql("""
SELECT max((dropOff_datetime - pickup_datetime) * 24) as longest_trip FROM
    fhv_data
""")

In [49]:
df_trip.show()

+--------------------+
|        longest_trip|
+--------------------+
|INTERVAL '631152 ...|
+--------------------+



In [50]:
df_zones = spark.read.parquet('zones/')

In [52]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [54]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [55]:
df_zones.registerTempTable('zones')

In [60]:
df_result = spark.sql("""
SELECT Zone, count(*) as no_trips FROM
    fhv_data
inner join zones
on PULocationID = LocationId
group by 1
order by 2 asc
""").show()

+--------------------+--------+
|                Zone|no_trips|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
|        Battery Park|      15|
|Saint Michaels Ce...|      23|
|Breezy Point/Fort...|      25|
|Marine Park/Floyd...|      26|
|        Astoria Park|      29|
|    Inwood Hill Park|      39|
|       Willets Point|      47|
|Forest Park/Highl...|      53|
|  Brooklyn Navy Yard|      57|
|        Crotona Park|      62|
|        Country Club|      77|
|     Freshkills Park|      89|
|       Prospect Park|      98|
|     Columbia Street|     105|
|  South Williamsburg|     110|
+--------------------+--------+
only showing top 20 rows

